In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
sys.path.insert(0, config.root_path)

In [3]:
import tensorflow as tf

from tensorflow.python import keras
import json
import pandas as pd
import numpy as np
import os

from src.encoders.context_encoder_ldabert_2 import ContextEncoder
from src.dataset.ldabert_2 import LDABERT2Dataset

## LDA BERT 2 Dataset

In [17]:
dataset = LDABERT2Dataset(dataset_type="clinical",
                       pct_data=0.001,
                          max_seq_length=128,
                       max_segment_length=5,
                       augment_pct=0.001,
                         split="train")

In [5]:
sentences, tokenized_sentences, labels = dataset.process()

In [7]:
lda_sentences, lda_token_lists, lda_new_labels = dataset.preprocess_lda(sentences, labels)

Preprocessing raw texts ...
sentences length 10
Preprocessing raw texts. Done!
lda sentences length 10


## Tri Encoder

In [21]:
context_encoder = ContextEncoder(final_dropout=0.5, dense_neurons=64, max_sentence_length=128, gamma=15)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['embeddings.position_ids']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [22]:
# test_data = [['This is an example sentence', 'This is a new sentence', 'And this is a continuation'], 
#              ['This is a new sentence', 'And this is a continuation', 'And another continuation'], 
#              ['And this is a continuation', 'And another continuation', 'And yet another continuation']]

In [25]:
context_encoder([[left_input, mid_input, right_input, lda_left_input, lda_mid_input, lda_right_input]])

IndexError: list index out of range

In [382]:
context_encoder.summary()

Model: "context_encoder_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
tf_bert_model_24 (TFBertMode multiple                  109482240 
_________________________________________________________________
dense_input_left (Dense)     multiple                  49856     
_________________________________________________________________
dense_input_mid (Dense)      multiple                  49856     
_________________________________________________________________
dense_input_right (Dense)    multiple                  49856     
_________________________________________________________________
dense_output (Dense)         multiple                  193       
_________________________________________________________________
final_dropout (Dropout)      multiple                  0         
Total params: 109,632,001
Trainable params: 109,632,001
Non-trainable params: 0
__________________________________

## Generate Vectors

In [17]:
dataset_types = ["clinical", "wiki", "fiction"]

for d in dataset_types:
    dataset = LDABERTDataset(dataset_type=d,
                           pct_data=1,
                           max_segment_length=5,
                           augment_pct=0.5)
    
    sentences, tokenized_sentences, labels = dataset.process(preprocess=True)
    
    # vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
    vectors_path = '../../data/{}_vectors/lda_bert_{}_{}_{}.pkl'.format(d, d, 
                                                                        dataset.pct_data, 
                                                                        dataset.augment_pct)

    saved_vectors, saved_tokens, saved_labels = dataset.get_vectors(vectors_path)

    if len(saved_vectors) == 0:
        saved_vectors, saved_tokens, saved_labels = dataset.create_vectors(vectors_path)

Preprocessing raw texts ...
10259 10259
Preprocessing raw texts. Done!
something went wrong [Errno 2] No such file or directory: 'C:\\Users\\Computer\\Google Drive\\SCHOOL\\PhD\\Code\\context-encoder-v2\\data\\generated_vectors\\clinical_vectors\\lda_bert_clinical_1_0.5.pkl'
root path C:\Users\Computer\Google Drive\SCHOOL\PhD\Code\context-encoder-v2
Getting vector representations for LDA ...
Getting vector representations for LDA. Done!
Getting vector representations for S_BERT ...


Batches:   0%|          | 0/316 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
[[ 1.5030304   0.6538147   1.2617971  ... -0.07280263  0.23336096
   0.8262265 ]
 [ 1.1767309   0.5364523   1.2677495  ...  0.11217795  0.09784786
  -0.03438686]
 [ 0.62225527  1.0551611   2.202089   ...  1.1347252   0.14224464
   0.05253705]
 ...
 [ 0.84182185  0.9099033   1.3536335  ...  0.15851161 -0.2311791
   0.06266249]
 [ 1.803962    1.047267    0.59083563 ...  0.8280431   0.17820705
   1.4119322 ]
 [ 0.9566428   0.61323845  0.62884104 ...  0.358712    0.46842542
   0.9512988 ]]
(10093, 768)
Fitting Autoencoder ...
Fitting Autoencoder Done!
Preprocessing raw texts ...
10316 10316
Preprocessing raw texts. Done!
something went wrong [Errno 2] No such file or directory: 'C:\\Users\\Computer\\Google Drive\\SCHOOL\\PhD\\Code\\context-encoder-v2\\data\\generated_vectors\\wiki_vectors\\lda_bert_wiki_1_0.5.pkl'
root path C:\Users\Computer\Google Drive\SCHOOL\PhD\Code\context-encoder-v2
Getting vector representations for LDA ...
Getting vect

Batches:   0%|          | 0/315 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
[[ 0.8427308   2.1591754   0.699877   ...  0.8665418   1.1667999
   0.4564934 ]
 [ 0.83219594  1.2316083   1.1702161  ...  0.49829528  0.6233119
   0.06990469]
 [ 0.3945257   1.6743623   0.38195902 ...  0.46317536  1.0000203
   0.16089   ]
 ...
 [ 0.76700145  1.1213465   0.20496714 ...  1.2895375   0.69258344
   0.6362469 ]
 [ 0.92033386  0.65964675  0.28187832 ...  0.70216185  0.95357805
   0.51770526]
 [ 0.36274368  0.16606128  1.0401337  ...  0.64025915  0.34550846
  -0.128405  ]]
(10055, 768)
Fitting Autoencoder ...
Fitting Autoencoder Done!
Preprocessing raw texts ...
15055 15055
Preprocessing raw texts. Done!
something went wrong [Errno 2] No such file or directory: 'C:\\Users\\Computer\\Google Drive\\SCHOOL\\PhD\\Code\\context-encoder-v2\\data\\generated_vectors\\fiction_vectors\\lda_bert_fiction_1_0.5.pkl'
root path C:\Users\Computer\Google Drive\SCHOOL\PhD\Code\context-encoder-v2
Getting vector representations for LDA ...
Getting 

Batches:   0%|          | 0/427 [00:00<?, ?it/s]

Getting vector representations for BERT. Done!
[[ 1.2645738   0.86432266  0.5193494  ... -0.02081079  0.5192658
   0.43456942]
 [ 0.8827983   0.71797484  0.20867829 ...  1.2551633   0.83528435
   0.05350147]
 [ 0.7183079   2.2732043   1.5674933  ...  0.41826937  1.3192451
   0.8833546 ]
 ...
 [ 1.0824481   0.60173595  1.460718   ...  0.67573845  0.58170366
   0.15544572]
 [ 0.739459    1.11267     1.5335777  ...  0.5901724   0.93928665
   0.7587939 ]
 [ 0.97557276  1.5640703   1.6278915  ...  0.54493403  0.67613524
   0.49542746]]
(13651, 768)
Fitting Autoencoder ...
Fitting Autoencoder Done!


In [9]:
dataset = LDABERTDataset(dataset_type="clinical",
                       pct_data=1,
                       max_segment_length=5,
                       augment_pct=1)

In [10]:
sentences, tokenized_sentences, labels = dataset.process(preprocess=False)

In [11]:
len(sentences)

16122

In [12]:
tokenized_sentences[:5]

[]

### Vectors

In [8]:
config.root_path

'C:\\Users\\Computer\\Google Drive\\SCHOOL\\PhD\\Code\\context-encoder-v2'

In [18]:
# vectors_path = '../data/clinical_vectors/lda_bert_{}_{}.pkl'.format(dataset_type, len(sentences))
vectors_path = '../../data/lda_bert_2/{}/{}_{}.pkl'.format(dataset.dataset_type, 
                                                        dataset.pct_data, 
                                                        dataset.augment_pct)

saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.get_saved_vectors(vectors_path)

if len(saved_vectors) == 0:
    saved_vectors, saved_labels, saved_sentences, saved_tokenized_sentences = dataset.create_vectors(vectors_path)

In [19]:
left_input, mid_input, right_input = dataset.format_sentences_tri_input_plus(saved_tokenized_sentences)
lda_left_input, lda_mid_input, lda_right_input = dataset.format_sentences_tri_input(saved_vectors)

In [20]:
left_input

{'input_ids': <tf.Tensor: shape=(6, 87), dtype=int32, numpy=
 array([[  101,  2065,  1037,  2711,  1005,  1055,  2310, 18674,  2668,
          4834,  2003,  6022,  2896,  2084,  2030,  5020,  2000,  2216,
          1997,  1037,  2711,  1005,  1055,  3671,  5505,  3446,  1010,
          2027,  2071,  2022,  4039,  2000,  2079,  2172,  2000,  7200,
          1998,  2947,  2097,  2025,  4503,  2151,  3255,  1012,   102,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0],
        [  101,  2019, 17577,  2015,  2089,  2022,  6219,  2241,  2006,
          2037,  3802, 20569,  1041,  1043, 19610,  4747, 21252, 19610,
          2953, 25032, 22974,  2278,  4385,  9413, 22123,  8093,  7361,
         10448, 16530, 

## Training

In [42]:
model = ContextEncoder(final_dropout=0.8,
                               dense_neurons=256,
                             lstm_size=256,
                             lstm_dropout_percentage=0.75,
                             cnn_filters=8,
                             cnn_kernel_size=3,
                             pool_size=2)

In [28]:
fake_output = model(tf.constant([[[1,2]],[[3,4]],[[5,6]]])) 

tf.Tensor(
[[[1 2]]

 [[3 4]]

 [[5 6]]], shape=(3, 1, 2), dtype=int32)
tf.Tensor([[1 2]], shape=(1, 2), dtype=int32)


InvalidArgumentError: in user code:

    C:\Users\Computer\Google Drive\SCHOOL\PhD\Code\context-encoder-v2\src\encoders\context_encoder.py:58 None  *
        lambda t: tf.reshape(t, [rows, cols]), elems=input_tensor)
    C:\Users\Computer\anaconda3\envs\phd\lib\site-packages\tensorflow\python\util\dispatch.py:201 wrapper  **
        return target(*args, **kwargs)
    C:\Users\Computer\anaconda3\envs\phd\lib\site-packages\tensorflow\python\ops\array_ops.py:195 reshape
        result = gen_array_ops.reshape(tensor, shape, name)
    C:\Users\Computer\anaconda3\envs\phd\lib\site-packages\tensorflow\python\ops\gen_array_ops.py:8372 reshape
        tensor, shape, name=name, ctx=_ctx)
    C:\Users\Computer\anaconda3\envs\phd\lib\site-packages\tensorflow\python\ops\gen_array_ops.py:8397 reshape_eager_fallback
        ctx=ctx, name=name)
    C:\Users\Computer\anaconda3\envs\phd\lib\site-packages\tensorflow\python\eager\execute.py:60 quick_execute
        inputs, attrs, num_outputs)

    Invalid

In [37]:
fake_output

<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[0.67855525]], dtype=float32)>

In [21]:
model.summary()

Model: "context_encoder_complex"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bi-directional_left (Bidirec multiple                  140288    
_________________________________________________________________
bi-directional_mid (Bidirect multiple                  140288    
_________________________________________________________________
bi-directional_right (Bidire multiple                  140288    
_________________________________________________________________
left_context (Attention)     multiple                  0         
_________________________________________________________________
mid_sentence (Attention)     multiple                  0         
_________________________________________________________________
right_context (Attention)    multiple                  0         
_________________________________________________________________
conv1D_left (Conv1D)         multiple      

ValueError: You tried to call `count_params` on dense_input_left, but the layer isn't built. You can build it manually via: `dense_input_left.build(batch_input_shape)`.

In [43]:
METRICS = [
      keras.metrics.BinaryAccuracy(name='accuracy')
]

In [44]:
EPOCHS = 300
BATCH_SIZE = 256

# balanced = balanced binary crossentropy
checkpoint_filepath = '../models/LDABERT/complex/{}-{}-{}-pct-{}-aug/checkpoint.ckpt'.format(
                        dataset.dataset_type,                    
                        len(sentences), 
                        dataset.pct_data,
                        dataset.augment_pct)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    save_best_only=False,
    mode="auto",
    save_freq="epoch")

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_accuracy', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

callbacks = [
#     early_stopping,
    model_checkpoint_callback
]

In [45]:
checkpoint_filepath

'../models/LDABERT/complex/clinical-16122-1-pct-1-aug/checkpoint.ckpt'

In [46]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
              loss=tf.keras.losses.BinaryCrossentropy(),
              metrics=METRICS)

In [47]:
try:
    model.load_weights(checkpoint_filepath)
except:
    print("No checkpoint available.")

In [50]:
history = model.fit([left_input, mid_input, right_input], 
                    tf.convert_to_tensor(saved_labels), 
                    epochs=EPOCHS,
                    validation_split=0.1,
                    batch_size=BATCH_SIZE,
                    verbose=1, 
#                     class_weight=class_weight,
                    callbacks=callbacks)

Epoch 1/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3372 - accuracy: 0.8526 - val_loss: 0.2992 - val_accuracy: 0.8714
Epoch 2/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3403 - accuracy: 0.8520 - val_loss: 0.3009 - val_accuracy: 0.8739
Epoch 3/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3368 - accuracy: 0.8545 - val_loss: 0.2997 - val_accuracy: 0.8726
Epoch 4/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3387 - accuracy: 0.8507 - val_loss: 0.3055 - val_accuracy: 0.8752
Epoch 5/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3401 - accuracy: 0.8508 - val_loss: 0.2999 - val_accuracy: 0.8689
Epoch 6/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3341 - accuracy: 0.8527 - val_loss: 0.3038 - val_accuracy: 0.8663
Epoch 7/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3382 - accuracy: 0.8514 - val_loss: 0.2996 - val_accuracy: 0.8739
Epoch 8/300
5

Epoch 59/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3231 - accuracy: 0.8599 - val_loss: 0.2919 - val_accuracy: 0.8701
Epoch 60/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3270 - accuracy: 0.8580 - val_loss: 0.2940 - val_accuracy: 0.8739
Epoch 61/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3284 - accuracy: 0.8574 - val_loss: 0.3038 - val_accuracy: 0.8638
Epoch 62/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3311 - accuracy: 0.8545 - val_loss: 0.2933 - val_accuracy: 0.8689
Epoch 63/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3257 - accuracy: 0.8572 - val_loss: 0.2938 - val_accuracy: 0.8739
Epoch 64/300
56/56 [==============================] - 0s 5ms/step - loss: 0.3256 - accuracy: 0.8571 - val_loss: 0.2958 - val_accuracy: 0.8745
Epoch 65/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3257 - accuracy: 0.8576 - val_loss: 0.3050 - val_accuracy: 0.8707
Epoch 

56/56 [==============================] - 0s 4ms/step - loss: 0.3071 - accuracy: 0.8668 - val_loss: 0.2868 - val_accuracy: 0.8758
Epoch 174/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3029 - accuracy: 0.8686 - val_loss: 0.2877 - val_accuracy: 0.8770
Epoch 175/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3081 - accuracy: 0.8708 - val_loss: 0.2901 - val_accuracy: 0.8777
Epoch 176/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3028 - accuracy: 0.8689 - val_loss: 0.2838 - val_accuracy: 0.8764
Epoch 177/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3045 - accuracy: 0.8679 - val_loss: 0.2830 - val_accuracy: 0.8745
Epoch 178/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3080 - accuracy: 0.8676 - val_loss: 0.2979 - val_accuracy: 0.8783
Epoch 179/300
56/56 [==============================] - 0s 4ms/step - loss: 0.3029 - accuracy: 0.8709 - val_loss: 0.2921 - val_accuracy: 0.8764
Epoch 180/300

UnknownError: Failed to rename: ../models/LDABERT/complex/clinical-16122-1-pct-1-aug\checkpoint.tmpd9a0d0c56e0e4831bd0e9c7ea405e7cd to: ../models/LDABERT/complex/clinical-16122-1-pct-1-aug\checkpoint : Access is denied.
; Input/output error

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

## Experiment

In [14]:
from utils.experiments import get_experiments_json, get_experiments, save_results

In [15]:
experiments_config = get_experiments_json('ldabert_simple_test')
epxeriments_config_df = pd.DataFrame.from_dict(experiments_config)
epxeriments_config_df

,bert_type,dataset_type,final_dropout,dense_neurons,pct_data,augment_pct,epochs
0,ldabert,clinical,0.5,64,1,0.5,1000
1,ldabert,clinical,0.5,64,1,1.0,1000
2,ldabert,clinical,0.5,256,1,0.5,1000
3,ldabert,clinical,0.5,256,1,1.0,1000
4,ldabert,wiki,0.5,64,1,0.5,1000
5,ldabert,wiki,0.5,64,1,1.0,1000
6,ldabert,wiki,0.5,256,1,0.5,1000
7,ldabert,wiki,0.5,256,1,1.0,1000
8,ldabert,fiction,0.5,64,1,0.5,1000
9,ldabert,fiction,0.5,64,1,1.0,1000


In [8]:
# Read local `config.toml` file.
config = get_experiments('ALBERT_FINETUNE_SIMPLE')
config_df = pd.DataFrame.from_dict(config)
config_df

,bert_type,dataset_type,finetune_bert,pct_data,augment_pct,epochs
0,albert,clinical,True,1,0.1,200
1,albert,clinical,True,1,0.5,200
2,albert,clinical,True,1,1.0,200
3,albert,fiction,True,1,0.1,200
4,albert,fiction,True,1,0.5,200
5,albert,fiction,True,1,1.0,200
6,albert,wiki,True,1,0.1,200
7,albert,wiki,True,1,0.5,200
8,albert,wiki,True,1,1.0,200


In [69]:
config_df.to_csv(r'../models/experiment.csv', header=None, index=None, sep=' ', mode='a')

In [59]:
for experiment in config:
    bert_type = config['bert_type']
    dataset_type = config['dataset_type']
    finetune_bert = config['finetune_bert']
    pct_data = config['pct_data']
    augment_pct = config['augment_pct']
    epochs = config['epochs']
    print("params:", bert_type, dataset_type, finetune_bert, pct_data, augment_pct, epochs)
    
    # init model
    print("initializing model...")
    model = ContextEncoder(final_dropout=0.5,
                           dense_neurons=64,
                           bert_trainable=finetune_bert,
                           bert_type="albert-base-v2")
    
    # init dataset
    print("initializing dataset...")
    dataset = AlbertDataset(dataset_type=dataset_type,
                           pct_data=pct_data,
                           max_segment_length=5,
                           augment_pct=augment_pct)
    
    # process dataset
    print("processing dataset...")
    sentences, tokenized_sentences, labels = dataset.process()
    
    # create checkpoint path
    checkpoint_filepath = '../models/ALBERT/finetune/simple/{}-{}-{}-pct-{}-aug/checkpoint'.format(
                            dataset_type,                    
                            len(sentences), 
                            pct_data,
                            augment_pct)
    print(checkpoint_filepath)
    
    # compiling model
    print("compiling the model...")
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-3),
                  loss=tf.keras.losses.BinaryCrossentropy(),
                  metrics=METRICS)
    
    try:
        model.load_weights(checkpoint_filepath)
        print("model loaded.")
    except:
        print("No checkpoint available.")
    
    # 
    print("starting the training process...")
    history = model.fit(dataset.format_sentences_tri_input(tokenized_sentences), 
                        tf.convert_to_tensor(labels), 
                        epochs=EPOCHS,
                        validation_split=0.1,
                        batch_size=BATCH_SIZE,
                        verbose=1, 
                        # class_weight=class_weight,
                        callbacks=callbacks)
    
    # assigning history to experiment object for saving.
    experiment["history"] = history
    
    print("saving results...")
    save_results(experiment)

9